In [2]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os
# U-Net Model
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        def CBR(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        
        self.enc1 = CBR(3, 64)
        self.enc2 = CBR(64, 128)
        self.enc3 = CBR(128, 256)
        self.enc4 = CBR(256, 512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.bottleneck = CBR(512, 1024)
        
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = CBR(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = CBR(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = CBR(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = CBR(128, 64)
        
        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)
        
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))
        
        # Decoder
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.dec4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.dec3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.dec2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.dec1(dec1)
        
        return torch.sigmoid(self.final_conv(dec1))

# Custom Dataset
class YUCARSDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.img_names = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        mask_path = os.path.join(self.mask_dir, self.img_names[idx])  # Assuming masks have the same name as images
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Assuming mask is a grayscale image

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [4]:
import torchvision

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load the dataset
dataset = torchvision.datasets.OxfordIIITPet(root='data', 
                                             split='trainval', 
                                             target_types='segmentation', 
                                             download=True, 
                                             transform=transform)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 791918971/791918971 [01:50<00:00, 7150582.80it/s]


Extracting data/oxford-iiit-pet/images.tar.gz to data/oxford-iiit-pet


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19173078/19173078 [00:04<00:00, 4320731.47it/s]


Extracting data/oxford-iiit-pet/annotations.tar.gz to data/oxford-iiit-pet


In [5]:

# Model, Loss, and Optimizer
model = UNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)


UNet(
  (enc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bottleneck): Sequential(
  

In [11]:
import torchvision.datasets as datasets
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images and masks to 128x128
    transforms.ToTensor(),          # Convert both images and masks to tensors
])

dataset = datasets.OxfordIIITPet(
    root='data', 
    split='trainval', 
    target_types='segmentation', 
    download=True,
    transform=transform,            # Apply transformations to the images
    target_transform=transform      # Apply the same transformations to the segmentation masks
)
# Create a DataLoader for batching and shuffling
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [18]:

from torch.utils.tensorboard import SummaryWriter

num_epochs = 200
writer = SummaryWriter('runs/unet_experiment')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        #print("test")
        outputs = model(images)
        
        #logits = outputs.logits
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        progress_bar.set_postfix(loss=running_loss/len(progress_bar))
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
    if (epoch + 1) % 50 == 0:
        # 
        torch.save(model.state_dict(), f'weights/oxford_pet_{epoch+1}.pth')
        writer.add_images('input_images', images, epoch)
        writer.add_images('segmentation_masks', labels, epoch)
        writer.add_images('predictions', torch.sigmoid(outputs), epoch)


print("Finished Training")

Epoch 1/200: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:20<00:00, 11.49batch/s, loss=0.0423]


Epoch [1/200], Loss: 0.042286320840534956


Epoch 2/200: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:19<00:00, 11.60batch/s, loss=0.0422]


Epoch [2/200], Loss: 0.042241935885470844


Epoch 3/200: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:19<00:00, 11.67batch/s, loss=0.0422]


...

Epoch [198/200], Loss: 0.04186818773007911


Epoch 199/200: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:25<00:00,  9.09batch/s, loss=0.0419]


Epoch [199/200], Loss: 0.04186819053214529


Epoch 200/200: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:21<00:00, 10.60batch/s, loss=0.0419]


Epoch [200/200], Loss: 0.04186819968340189
Finished Training
